In [1]:
include("../../../julia/FNC_init.jl")

  Activating 

project at `~/Documents/GitHub/fnc`


[**Demo %s**](#demo-house-qr)


We will use Householder reflections to produce a QR factorization of a random matrix.
```{tip}
:class: dropdown
The `rand` function can select randomly from within the interval $[0,1]$, or from a vector or range that you specify.
```

In [2]:
A = rand(float(1:9), 6, 4)
m,n = size(A)

(6, 4)

```{index} Julia; normalize, ! Julia; I
```

Our first step is to introduce zeros below the diagonal in column 1 by using {eq}`hhvector` and {eq}`hhreflect`. 
:::
```{tip}
:class: dropdown
`I` can stand for an identity matrix of any size, inferred from the context when needed.
```

In [3]:
z = A[:, 1];
v = normalize(z - norm(z) * [1; zeros(m-1)])
P₁ = I - 2v * v'   # reflector

6×6 Matrix{Float64}:
 0.40179    0.53572     0.066965    0.334825   0.602685    0.26786
 0.53572    0.520243   -0.0599696  -0.299848  -0.539727   -0.239878
 0.066965  -0.0599696   0.992504   -0.037481  -0.0674658  -0.0299848
 0.334825  -0.299848   -0.037481    0.812595  -0.337329   -0.149924
 0.602685  -0.539727   -0.0674658  -0.337329   0.392808   -0.269863
 0.26786   -0.239878   -0.0299848  -0.149924  -0.269863    0.880061

We check that this reflector introduces zeros as it should:

In [4]:
P₁ * z

6-element Vector{Float64}:
 14.933184523068077
  1.5543122344752192e-15
  1.942890293094024e-16
  1.3322676295501878e-15
  1.1102230246251565e-15
  8.881784197001252e-16

Now we replace $\mathbf{A}$ by $\mathbf{P}\mathbf{A}$.

In [5]:
A = P₁ * A

6×4 Matrix{Float64}:
 14.9332        9.70992    9.30813    14.4644
  1.55431e-15  -0.217907   0.455298   -3.16236
  1.94289e-16   3.47276    5.18191     4.6047
  1.33227e-15   4.36381   -0.0904386  -1.97648
  1.11022e-15  -0.745146  -6.36279    -3.55766
  8.88178e-16  -0.108954   5.72765     1.41882

We are set to put zeros into column 2. We must not use row 1 in any way, lest it destroy the zeros we just introduced. So we leave it out of the next reflector.

In [6]:
z = A[2:m, 2]
v = normalize(z - norm(z) * [1; zeros(m-2)])
P₂ = I - 2v * v'

5×5 Matrix{Float64}:
 -0.0386921   0.616631    0.774847   -0.13231     -0.0193461
  0.616631    0.63393    -0.459997    0.0785472    0.011485
  0.774847   -0.459997    0.421976    0.0987009    0.0144319
 -0.13231     0.0785472   0.0987009   0.983146    -0.00246432
 -0.0193461   0.011485    0.0144319  -0.00246432   0.99964

We now apply this reflector to rows 2 and below only.

In [7]:
A[2:m, :] = P₂ * A[2:m, :]
A

6×4 Matrix{Float64}:
 14.9332        9.70992       9.30813  14.4644
  9.27893e-16   5.63183       3.83869   1.87356
  5.6617e-16   -1.00692e-15   3.17332   1.61507
  1.79957e-15  -6.74785e-16  -2.61439  -5.73319
  1.03043e-15   1.99563e-16  -5.93181  -2.91618
  8.76511e-16   2.02104e-17   5.79067   1.51262

We need to iterate the process for the last two columns.

In [8]:
for j in 3:n
    z = A[j:m, j]
    v = normalize(z - norm(z) * [1; zeros(m-j)])
    P = I - 2v * v'
    A[j:m, :] = P * A[j:m, :]
end

We have now reduced the original to an upper triangular matrix using four orthogonal Householder reflections:

In [9]:
R = triu(A)

6×4 Matrix{Float64}:
 14.9332  9.70992  9.30813  14.4644
  0.0     5.63183  3.83869   1.87356
  0.0     0.0      9.25328   4.98971
  0.0     0.0      0.0       4.62307
  0.0     0.0      0.0       0.0
  0.0     0.0      0.0       0.0